In [93]:
from pyomo.environ import *

In [101]:
def init_power(model,i):
  return P[i-1]
def init_demand(model,i):
  return d[i-1]
def init_loss(model,i,j):
  return L[i-1][j-1]

In [102]:
from pyomo.environ import *

P = [280,300,440]
d = [200, 360, 380]
L = [[0, 0.5, 0.25],
     [0.5, 0, 0.15],
     [0.25, 0.15, 0]]
n=3

# create a model
model = ConcreteModel()

# declare parameters and sets
model.n = n
model.loc = Set(initialize=range(1,model.n+1))
model.demand = Param(model.loc, initialize=init_demand)
model.power = Param(model.loc, initialize=init_power)
model.loss = Param(model.loc, model.loc, initialize=init_loss)
model.x = Var(model.loc, model.loc, domain=Reals)

In [103]:
# declare objective
model.obj = Objective(expr=sum(abs(model.x[i, j]) * model.loss[i, j] for i in model.loc for j in model.loc))

In [104]:
# Assignment Constraints
model.assignment = ConstraintList()
for i in model.loc:
    for j in model.loc:
        model.assignment.add(expr=(model.x[i, j] + model.x[j, i] )== 0)
    trans = model.power[i] - model.demand[i] - sum(model.x[i, j] for j in model.loc)
    model.assignment.add(expr = (trans) >=0)
model.pprint()

4 Set Declarations
    assignment_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    loc : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    loss_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain  : Size : Members
        None :     2 : loc*loc :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain  : Size : Members
        None :     2 : loc*loc :    9 : {(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3)}

3 Param Declarations
    demand : Size=3, Index=loc, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   200
          2 :   360
          3 :   380
    loss : Size=9, Index=loss_index, Domain=Any, Default=None, Mutabl

In [105]:
solver = SolverFactory('ipopt.exe')
solver.solve(model)

# display solution
print('\nObjective Function = ', model.obj())
print('\nDecision Variables')


Objective Function =  0.0

Decision Variables


In [106]:
print( model.x.display())

x : Size=9, Index=x_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :  None :   0.0 :  None : False : False :  Reals
    (1, 2) :  None :   0.0 :  None : False : False :  Reals
    (1, 3) :  None :   0.0 :  None : False : False :  Reals
    (2, 1) :  None :   0.0 :  None : False : False :  Reals
    (2, 2) :  None :   0.0 :  None : False : False :  Reals
    (2, 3) :  None :   0.0 :  None : False : False :  Reals
    (3, 1) :  None :   0.0 :  None : False : False :  Reals
    (3, 2) :  None :   0.0 :  None : False : False :  Reals
    (3, 3) :  None :   0.0 :  None : False : False :  Reals
None
